# Train-test split
Train-test split is a utility to create training and testing set as subsets from a single data set.

Train-test split was added in MADlib 1.12.

In [15]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [17]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12, git revision: rel/v1.11-57-gd5d5a26, cmake configuration time: Sun Aug 20 19:18:30 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create input table

In [43]:
%%sql 
DROP TABLE IF EXISTS test;

CREATE TABLE test(
    id1 INTEGER,
    id2 INTEGER,
    gr1 INTEGER,
    gr2 INTEGER
);

INSERT INTO test VALUES
(1,0,1,1),
(2,0,1,1),
(3,0,1,1),
(4,0,1,1),
(5,0,1,1),
(6,0,1,1),
(7,0,1,1),
(8,0,1,1),
(9,0,1,1),
(9,0,1,1),
(9,0,1,1),
(9,0,1,1),
(0,1,1,2),
(0,2,1,2),
(0,3,1,2),
(0,4,1,2),
(0,5,1,2),
(0,6,1,2),
(10,10,2,2),
(20,20,2,2),
(30,30,2,2),
(40,40,2,2),
(50,50,2,2),
(60,60,2,2),
(70,70,2,2);

SELECT * FROM test ORDER BY id1, id2, gr1, gr2;

Done.
Done.
25 rows affected.
25 rows affected.


id1,id2,gr1,gr2
0,1,1,2
0,2,1,2
0,3,1,2
0,4,1,2
0,5,1,2
0,6,1,2
1,0,1,1
2,0,1,1
3,0,1,1
4,0,1,1


# 2.  Sample without replacement

In [44]:
%%sql
DROP TABLE IF EXISTS out;

SELECT madlib.train_test_split(
                                'test',    -- Source table
                                'out',     -- Output table
                                0.5,       -- Sample proportion
                                0.5,       -- Sample proportion
                                'gr1,gr2', -- Strata definition
                                'id1,id2', -- Columns to output
                                FALSE,     -- Sample without replacement
                                FALSE);    -- Do not separate output tables

SELECT * FROM out ORDER BY split,gr1,gr2,id1,id2;

Done.
1 rows affected.
25 rows affected.


gr1,gr2,id1,id2,split
1,1,2,0,0
1,1,4,0,0
1,1,5,0,0
1,1,7,0,0
1,1,8,0,0
1,1,9,0,0
1,2,0,3,0
1,2,0,5,0
1,2,0,6,0
2,2,10,10,0


# 3.  Sample with replacement

In [47]:
%%sql
DROP TABLE IF EXISTS out, out_train, out_test;

SELECT madlib.train_test_split(
                                'test',    -- Source table
                                'out',     -- Output table
                                0.5,       -- train_proportion
                                NULL,      -- Default = 1 - train_proportion = 0.5
                                'gr1,gr2', -- Strata definition
                                'id1,id2', -- Columns to output
                                TRUE,      -- Sample with replacement
                                TRUE);     -- Separate output tables

SELECT * FROM out_train ORDER BY gr1,gr2,id1,id2;

Done.
1 rows affected.
12 rows affected.


gr1,gr2,id1,id2
1,1,1,0
1,1,6,0
1,1,9,0
1,1,9,0
1,1,9,0
1,1,9,0
1,2,0,2
1,2,0,3
1,2,0,5
2,2,20,20


In [48]:
%%sql
SELECT * FROM out_test ORDER BY gr1,gr2,id1,id2;

13 rows affected.


gr1,gr2,id1,id2
1,1,1,0
1,1,4,0
1,1,5,0
1,1,7,0
1,1,8,0
1,1,9,0
1,2,0,1
1,2,0,4
1,2,0,4
2,2,30,30


In [7]:
%%sql
DROP TABLE IF EXISTS out;

SELECT madlib.train_test_split(
                                'test',    -- Source table
                                'out',     -- Output table
                                0.5    -- Sample proportion
);
SELECT * FROM out ORDER BY split,gr1,gr2,id1,id2;

Done.
1 rows affected.
25 rows affected.


id1,id2,gr1,gr2,split
2,0,1,1,0
3,0,1,1,0
5,0,1,1,0
7,0,1,1,0
9,0,1,1,0
9,0,1,1,0
9,0,1,1,0
0,4,1,2,0
0,6,1,2,0
10,10,2,2,0
